In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import load_model
from keras import backend as K

from utils import load_test_data

Using TensorFlow backend.


In [2]:
ensembling_models = {
    'DenseNet201-Fine-Tune': {'img_size': 224},
    'Xception-Fine-Tune': {'img_size': 299}
}

In [3]:
for key in ensembling_models.keys():
    print(key)
    img_size = ensembling_models[key]['img_size']
    model = load_model('./saved_models/{}.h5'.format(key))
    X_test, X_id = load_test_data(Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)

DenseNet201-Fine-Tune


ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (0, 1)

In [4]:
num_models = float(len(ensembling_models.keys()))
y_test_pred_prob = 0
for key in ensembling_models.keys():
    y_test_pred_prob += ensembling_models[key]['y_test_pred_prob']

y_test_pred_prob /= num_models

In [5]:
y_test_pred = y_test_pred_prob.argmax(axis=-1)

In [6]:
import numpy as np
import pandas as pd
X_test, X_id = load_test_data(Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
y_test_pred_df = pd.DataFrame({'id': np.array(X_id), 'class':y_test_pred}).sort_values(by='id')

In [7]:
y_test_pred_df.to_csv('./submissions/ensemble.csv', index=False)